# RuGPT-3 (zero-shot / few-shot)

### Введение

**Из описания модели (ChatGPT + DeepSeek + habr.com)**

Для решения конкретной прикладной задачи дообучать модель не требуется, достаточно уже обученной модели подать на вход текст из нескольких примеров с разметкой и попросить продолжить последний.
Нейросеть обучена на русскоязычных текстах разных стилей: на энциклопедиях, социальных сетях, художественной и бизнес-литературе. С помощью такой модели можно выстроить с нуля даже рекомендательную систему — попробуйте продолжить с помощью модели фразу «Физик Максим, 28 лет, больше всего любит фильмы»

* Zero-shot — модель решает задачу без примеров, только по инструкции.
* Few-shot — модель решает задачу, увидев несколько примеров прямо в тексте запроса.

Это два ключевых метода инференса, которые позволяют использовать предобученную модель без ее дообучения (fine-tuning).

В обоих случаях веса модели НЕ меняются. Это не является обучением модели.

**Zero-Shot (Нулевой пример)**

Как работает: Вы даете модели только инструкцию или задачу на естественном языке, без каких-либо примеров.\
Что делает модель: Она должна понять саму суть задачи, опираясь только на свои знания, полученные при предобучении.\
Аналогия: Вы просите человека, который никогда не решал конкретных головоломок: «Переведи это предложение на французский язык». Он полагается на общее знание языков.

Пример промта (запроса):

```text
-------------
Классифицируй тональность следующего отзыва на продукт: "Я в восторге от новой камеры, качество съемки просто потрясающее!"

Ответ (позитивный/негативный/нейтральный):
-------------
```

**Few-Shot (Несколько примеров)**

Как работает: Вы даете модели несколько примеров (обычно 2-10) «входных данных -> правильный ответ», а затем — новую, целевую задачу.\
Что делает модель: Она учится на лету (in-context learning), улавливая паттерн из примеров: что требуется сделать, в каком формате выводить ответ.\
Аналогия: Вы показываете человеку 3 примера решения головоломки, а затем даете новую. Он выводит правило из примеров и применяет его.

Пример промта:

```text
-------------
Отзыв: "Батарея садится за пару часов, очень разочарован." | Тональность: негативный
Отзыв: "Нормальный телефон за свои деньги, ничего особенного." | Тональность: нейтральный
Отзыв: "Отличное приложение, все интуитивно понятно!" | Тональность: позитивный

Теперь классифицируй: "Доставка заняла две недели, хотя обещали за три дня."
Тональность:
-------------
```

**RuGPT-3**

* видел огромное количество корректного русского текста
* видел инструкции, пояснения, рассуждения
* умеет продолжать шаблоны “вопрос → ответ”

Он не классификатор, но:

* умеет вербально рассуждать о языке
* умеет следовать инструкциям
* умеет делать implicit classification

То есть мы не учим модель, а правильно спрашиваем.

У ruGPT-3 достаточно большой объём контекста - **2048 токенов**, что позволяет подавать на вход модели достаточно длинные тексты, а также несколько эталонных примеров + собственно пример, который модели нужно корректно обработать. 

LLM модели решают задачи классификации через генерацию текста. То есть вместо подхода \
"model" -> "logits" -> "argmax" -> "класс"\
используется \
"модель генерирует текст" -> "из текста извлекается класс".

### Подготовка

In [1]:
# preload local libs
import sys
import os
import pathlib

project_dir = str(pathlib.Path(os.getcwd()).parent.resolve())
if project_dir not in sys.path:
    sys.path.insert(1, project_dir)

# project_dir

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
import pandas as pd
import torch
from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef

from transformers import AutoTokenizer, GPT2LMHeadModel

import warnings
warnings.filterwarnings("ignore")

In [4]:
DEVICE = 'cuda:0'

#### Imports

In [5]:
# Загрузка RuGPT3. Вариант с HF.
RUGPT3_MODEL_NAME = "sberbank-ai/rugpt3medium_based_on_gpt2"

BATCH_SIZE = 32

In [6]:
train_df = pd.read_csv(project_dir + "/hw-08/RuCoLA/data/in_domain_train.csv")
test_df  = pd.read_csv(project_dir + "/hw-08/RuCoLA/data/in_domain_dev.csv")

train_df, val_df = train_test_split(
    train_df,
    test_size=0.2,
    random_state=1,
    stratify=train_df['acceptable']
)

# Так как дальше в Dataloader работаем с индексами строк, то после стратификации нужно пересобрать индексы
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

#### Модель и токенизатор

In [ ]:
rugpt3_tokenizer = AutoTokenizer.from_pretrained(RUGPT3_MODEL_NAME)
rugpt3_model = GPT2LMHeadModel.from_pretrained(RUGPT3_MODEL_NAME).to(DEVICE)
rugpt3_model.eval()

In [8]:
# Few-shot генерация
def generate_answer(prompt, max_new_tokens=3):
    inputs = rugpt3_tokenizer(prompt, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        outputs = rugpt3_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens, # Сколько токенов модель может ДОБАВИТЬ. Это НЕ длина всего текста, а длина продолжения. В нашем случае хватило бы одного
            temperature=1.0, # Это параметр случайности. softmax(logits / temperature). temperature работает ТОЛЬКО если do_sample=True
            do_sample=False, # Это переключатель стратегии генерации. 
                             # False - используется greedy decoding (берётся самый вероятный токен). 
                             # True - Используется sampling: токен выбирается случайно, но с учётом вероятностей
            #no_repeat_ngram_size=3, # Запрещает повторять n-граммы. "3 - нельзя повторить фразу из 3 слов"
            repetition_penalty=1.2, # Борется с зацикливанием.
            #top_k=5,    # Оставляет только K лучших токенов. "выбираем из 5 вероятных слов"
            #top_p=0.9,  # Берёт минимальное множество токенов с суммарной вероятностью p. "берем токены пока сумма вероятностей ≤ 0.9"
        )

    # GPT возвращает: prompt + продолжение
    text = rugpt3_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Оставляем только ответ
    text = text[len(prompt):].strip().lower()
    return text

#### Шаблоны

In [9]:
# Несколько вариантов затравок.

# Затравка 1
def prompt_template_1(sentence):
    return f"""Текст:
{sentence}

Вопрос: этот текст является грамматически правильным?

ответ:"""

# Затравка 2
def prompt_template_2(sentence):
    return f"""Текст:
{sentence}

Вопрос: текст является лингвистически корректным?

ответ:"""

# Затравка 3
def prompt_template_3(sentence):
    return f"""Предложение:
{sentence}

Вопрос: данное предложение является синтаксически правильным?

ответ:"""

# Затравка 4
def prompt_template_4(sentence):
    return f"""Предложение:
{sentence}

Вопрос: данное предложение является семантическии корректным?

ответ:"""

# Затравка 5
def prompt_template_5(sentence):
    return f"""Предложение:
{sentence}

Вопрос: данное предложение является морфологическии корректным?

ответ:"""

# Затравка 6
def prompt_template_6(sentence):
    return f"""Предложение:
{sentence}

Вопрос: корректно?

ответ:"""

#### Эксперименты

RuGPT-3 обучалась на большом количестве русских текстов. Скорее всего литературных. Поэтому формулировку вопросов, вероятнее всего, формулировать нужно литературно корректно.

В этой задаче я планирую спрашивать модель о грамматической корректности предложений. Как для этого лучше сформулировать вопрос. Понятие "acceptable" при оценке предлжений скорее всего не употреблялось, и в поисковых ответах чаще встречается для этого выражение "грамматически правильное" к оценке предложений. 

Проведу небольшой эксперимент, который позволит чуток понять логику работы механизма few-shot.

Я прдложу модели утверждение о грамматической правильности нескольких предложений и следом задам вопрос про эти же самые предложения. Посмотрим, как модель учитывает утверждения.

In [10]:
for _, row in train_df.sample(20).iterrows():
    var = 'грамматически правильным.'
    var_type = ''

    if row.acceptable == 0:
        var = 'грамматически неправильным.'
        if row.error_type == 'Syntax':
            var_type = ' В нем синтаксическая ошибка.'
        if row.error_type == 'Semantics':
            var_type = ' В нем  семантическая ошибка.'
        if row.error_type == 'Morphology':
            var_type = ' В нем  морфологическая ошибка.'

    template = f"""Предыдущий анализ: Предложение «{row.sentence}» является {var}{var_type}

Новый вопрос: Теперь проанализируй по аналогии следующее предложение: «{row.sentence}». Является ли это предложение грамматически правильным?

ответ:"""
    print(f'{template}')
    answer = generate_answer(template, max_new_tokens=20)
    print(f'Ответ GPT: {answer}')
    print('', '--', '', sep="\n")

Предыдущий анализ: Предложение «Принесет ли это мне какую-нибудь пользу?» является грамматически правильным.

Новый вопрос: Теперь проанализируй по аналогии следующее предложение: «Принесет ли это мне какую-нибудь пользу?». Является ли это предложение грамматически правильным?

ответ:
Ответ GPT: да, оно отвечает на поставленный выше вопрос.

вопрос 2. «какое из

--

Предыдущий анализ: Предложение «Но я считаю нелишним сохранить его здесь в том виде, как он был рассказан мне Владимиром Галактионовичем 24 июня 1910 года.» является грамматически правильным.

Новый вопрос: Теперь проанализируй по аналогии следующее предложение: «Но я считаю нелишним сохранить его здесь в том виде, как он был рассказан мне Владимиром Галактионовичем 24 июня 1910 года.». Является ли это предложение грамматически правильным?

ответ:
Ответ GPT: да, оно правильно.

вопросы для самоконтроля: 1) как ты думаешь,

--

Предыдущий анализ: Предложение «Петрович провозился с шинелью всего две недели.» является граммати

После проведения нескольких разных экспериментов с разными вариантами формулирования промта и разными предложениями из обучающей выборки получился очень неуиешительный результат. \
Модель не хочет использовать подсказки для формирования правильного ответа.

Подобный эксперимент был проведен с Deepseek и получен аналогичный результат. Я попросил обхяснить Deepseek почему такое произошло. Вот ответ:


Причина в том, что ваш промпт создал внутреннее противоречие, которое я, как модель, должен был разрешить. И разрешил его в пользу фактов и языковых норм, а не слепого следования шаблону.

Вот как это работает «под капотом»:

1. Конфликт инструкций: В вашем промпте было две взаимоисключающие инструкции:

   * Инструкция 1 (явная): «Проанализируй по аналогии» — то есть повтори структуру предыдущего анализа, где утверждалось, что предложение неправильное.
   * Инструкция 2 (неявная, но более фундаментальная): «Является ли это предложение грамматически правильным?» — то есть проведи фактический лингвистический анализ.

2. Приоритет истины и полезности: Когда модель сталкивается с таким конфликтом, её внутренние механизмы (обученные на огромных массивах текстов и правил) стремятся дать правдивый и полезный ответ, а не просто механически выполнить противоречивую инструкцию. Слепое повторение «Предыдущего анализа» («это неправильно») привело бы к распространению фактической ошибки, что снижает полезность и доверие.

3. Контекст — не догма: «Предыдущий анализ» в промпте — это всего лишь контекст, который может быть как верным, так и ошибочным. Моя задача — использовать его для понимания формата ответа (объяснить, исправить), но не для бездумного копирования содержания, если оно противоречит знаниям.

Что я сделал на практике:

Я принял ваш «Предыдущий анализ» как пример структуры ответа (сначала вердикт, потом объяснение, потом правило).

Однако я **не принял его фактическое утверждение как истину**, потому что мой тренировочные данные содержат информацию о том, что предложение «МГУ отпраздновало свой юбилей» соответствует современной языковой норме.

Поэтому я выполнил инструкцию «проанализируй» буквально — провёл самостоятельный анализ и дал результат, основанный на знаниях, а не на вводящем в заблуждение контексте.

**Вывод для создания промптов:**
Это важный урок. Если бы вашей целью было именно слепое копирование формата (например, для тестирования системы), нужно было бы сделать контекст нейтральным или явно указать на гипотетичность:

    «Предположим, что в предыдущем анализе было утверждение: "Предложение Х — неправильное". Теперь, чисто гипотетически и следуя только формату, проанализируй по аналогии предложение Y...»

В вашем же промпте более сильной оказалась имплицитная команда «проанализируй и дай истинный ответ», что, на мой взгляд, является более правильным и полезным поведением ассистента.
```


#### Zero/Few-shot

Формальное решение задачи

In [ ]:
# Few-shot контекст

def build_few_shot_context(df, n):
    examples = df.sample(n)

    context = ""
    for _, row in examples.iterrows():
        label = "грамматически правильно" if row.acceptable == 1 else "грамматически неправильно"
        context += f"""
Предложение: {row.sentence}
Ответ: {label}
"""
    return context

In [12]:
# Парсинг ответа GPT
# GPT иногда начинает фантазировать, поэтому чистим:

def parse_prediction(text, prompt):
    tf = text.split(' ')[0].strip('.,!')

    if tf == 'да':
        return 1
    if tf == 'нет':
        return 0
    
    # print(f'PROMT: {prompt}')
    print(f'ОТВЕТ НЕ ЗНАЮ: [{text}]')
    return 0
    # sys.exit(1)    

# Основной эксперимент

# Перебор:
# * prompt варианта
# * 0/1/2/4 shot

def evaluate_gpt(test_df, train_df, prompt_func, shots):
    preds = []
    labels = []

    for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
        context = build_few_shot_context(train_df, shots) if shots > 0 else ""
        # print('КОНТЕКСТ: ', context)

        prompt = context + "\n" + prompt_func(row.sentence)
        # print(f'ЗАПРОС: [[[ {prompt} ]]]')

        answer = generate_answer(prompt)
        # print(f'ОТВЕТ: [[[ {answer} ]]]')
        
        pred = parse_prediction(answer, prompt)

        preds.append(pred)
        labels.append(row.acceptable)

    return {
        "acc": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "mcc": matthews_corrcoef(labels, preds)
    }


In [ ]:
# Перебор вариантов

prompt_variants = (
    prompt_template_1,
    prompt_template_2,
    prompt_template_3,
    prompt_template_4,
    prompt_template_5,
    prompt_template_6,
)

shots_list = [0, 1, 2, 4]

results = []

for num, pfunc in enumerate(prompt_variants):
    for shots in shots_list:
        metrics = evaluate_gpt(test_df, train_df, pfunc, shots)
        results.append({
            "prompt": num,
            "shots": shots,
            **metrics
        })
        print(f'PROMT {num}', shots, metrics)

In [14]:
# Таблица результатов
pd.DataFrame(results)

,prompt,shots,acc,f1,mcc
0,0,0,0.517803,0.627943,-0.016018
1,0,1,0.280773,0.090090,0.024744
2,0,2,0.274669,0.058124,0.062115
3,0,4,0.279756,0.080519,0.041854
4,1,0,0.302136,0.230942,-0.098730
5,1,1,0.265514,0.037333,0.023716
6,1,2,0.274669,0.060606,0.052588
7,1,4,0.281790,0.080729,0.061788
8,2,0,0.743642,0.852804,-0.000635
9,2,1,0.431333,0.471145,0.036779


Учитывая величину MCC можно сказать, что модель оказалась необучаемая по данной задаче.